# Extract visual IDs from Power BI report

This notebook is an example of how the Power BI Embedded APIs can be used to gather the VisualIDs from a Power BI report. The script connects to a report end embeds it into the Notebook, then it loops through the pages of the reports and extracts visual information, which are then exported to JSON. 

The visual ID's can e.g. be used together with the logs from the Log Analytics integration, to correlate query events with specific report pages and visuals. 

## Notes

- The notebook was developed to be executed locally and currently doesn't work in Microsoft Fabric. 
- Execute the cells one-by-one. The report needs to be visibly embedded within the notebook before you can execute the last cell to collect the visual IDs.
- Each page needs to have at least one visual, as the loop otherwise throws an error.
- Please make sure that [powerbiclient](https://learn.microsoft.com/en-us/javascript/api/overview/powerbi/powerbi-jupyter) is installed.

In [ ]:
##################
# Authentication #
##################

from powerbiclient import Report
from powerbiclient.authentication import InteractiveLoginAuthentication

# Initiate interactive authentication. A browser window will open and allow you to log in with your Power BI User
auth = InteractiveLoginAuthentication()

In [2]:
##############################
# Embed report into notebook #
##############################

# Power BI Workspace and Report IDs - to replace with correct report information
workspace_id = ""
report_id = ""

# Embed report using auth
report = Report(group_id=workspace_id, report_id=report_id, auth=auth)

# Call report variabe to live-embed report into notebook
report

Report()

In [3]:
#######################################
# Collect page and visual information #
#######################################

# NOTE: Please make sure the report is visibly embedded in the notebook, before executing this cell

import json

# Function to get all visuals on a page and return them in a JSON format including the page they are located on
def get_visuals(self):
    # Get list of pages
    pages = self.get_pages()

    # Create string variable that will store the JSON data
    all_visuals = ""

    # Iterate through all the pages
    # NOTE: Each page will need at least 1 visual, otherwise the loop will not complete and throw an error
    for page in pages:
        # Load all visuals on current page in JSON format
        visuals = report.visuals_on_page(page['name'])

        # Loop through visuals and add current report page name to visuals info
        for visual in visuals:
            visual["ReportPage"] = page['displayName']

            # Save the current visuals back to the JSON variable, separated by a colon
            if len(all_visuals) > 1: all_visuals += ", "
            
            # Append the current visual to the JSON variable, by replacing all single with double quotes, as the JSON cannot be read correctly otherwise
            all_visuals += str(visual).replace(" '", ' "').replace("',", '",').replace("':", '":').replace("'}", '"}').replace("{'", '{"')
        
    # Return the full JSON as text and add brackets to make it a valid format as it is an array of separate JSONs
    return "[" + all_visuals + "]"

# Execute function
report_visuals = get_visuals(report)

# Replace with the folder path that you want to store the exported JSON file. End with "/"
path = 'C:/path/'
# Replace with file name
report_name="file_name"

with open(path+report_name+'.json', 'w') as f:
    json.dump(report_visuals, f)